# Exploratory Data Analysis Project Group 8

### Library Importation

In [1]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Import and Clean

This section's goal is to import different data sheet and clean them to make them meet our expectation of data and be ready for merging.

mbti data sheet

In [2]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

# MBTI data import
df_mbti = pd.read_csv("./data/raw/mbti.csv")

# Drop unrelated comlumns "stat", "enneagram", "img_url"
df_mbti = df_mbti.drop(["stat","enneagram","img_url"],axis = 1)

# Drop duplicated rows
df_mbti = df_mbti.drop_duplicates()

# Check if there is any NaN value or abnormal values in mbti columns 
print(df_mbti["mbti"].unique())
print(df_mbti["mbti"].isna().any())

# Drop the rows are "XXXX" (which is not a mbti type)
df_mbti = df_mbti[df_mbti["mbti"]!= "XXXX"]

# Make sure first letters are capitalized in "role" and "movie" columns
df_mbti["role"] = df_mbti["role"].str.title()
df_mbti["movie"] = df_mbti["movie"].str.title()

# Use str.strip() to remove spaces.
df_mbti["movie"] = df_mbti["movie"].str.strip()

# Regular expression pattern for extracting year: '(.*) \((\d{4})\)'
pattern_year = r'(.*) \((\d{4})\)'

# Extract movies with year
# Save the year into "release_year" columns
df_mbti[['movie_clean', 'release_year']] = df_mbti['movie'].str.extract(pattern_year)

# Regular expression pattern to match content in parentheses
pattern_parentheses = r'\s*\([^)]*\)'

# Remove content in parentheses
# Save movie names without parentheses into "movie_clean" columns
df_mbti['movie_clean'] = df_mbti['movie'].str.replace(pattern_parentheses, '', regex=True)

# Convert "release_year" column to datetime
df_mbti['release_year'] = pd.to_datetime(df_mbti['release_year'], format = '%Y').dt.year

# Drop "movie_name" column
df_mbti.drop("movie",axis=1, inplace=True)

# Rename columns to have consistency in all data sheets
df_mbti.rename(columns={'role': 'character','movie_clean':'movie_name'}, inplace=True)

df_mbti.to_csv("./data/cleaned/df_mbti.csv")

df_mbti.info()

['ESFP' 'XXXX' 'ESFJ' 'ESTP' 'ESTJ' 'ENFP' 'ENFJ' 'ENTP' 'ENTJ' 'ISFP'
 'ISFJ' 'ISTP' 'ISTJ' 'INFP' 'INFJ' 'INTP' 'INTJ']
False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16727 entries, 0 to 18740
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mbti          16727 non-null  object 
 1   character     16727 non-null  object 
 2   movie_name    16727 non-null  object 
 3   release_year  10939 non-null  float64
dtypes: float64(1), object(3)
memory usage: 653.4+ KB


In [3]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

df_mbti.sample(10)

,mbti,character,movie_name,release_year
135,ESFP,Donnie Azoff,The Wolf Of Wall Street,2013.0
9536,ENTJ,Monsieur Homais,Madame Bovary,2014.0
5936,ENFP,Daisy Fuller,The Curious Case Of Benjamin Button,2008.0
18695,INTJ,Dave Forbes,Fallen Series,NaN
15199,INFP,Quasimodo,The Hunchback Of Notre Dame,1996.0
18320,INTJ,Dr. William Chester Minor,The Professor And The Madman,2019.0
7610,ENTP,George Weasley,Harry Potter,NaN
12088,ISFJ,Erich,Midnight Express,1978.0
6256,ENFP,Maureen,The Meyerowitz Stories,2017.0
9963,ISFP,Johnny,God'S Own Country,NaN


imdb_movies

In [4]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

# imdb_movies data import
df_imdb_map = pd.read_csv("./data/raw/imdb_movies.csv")

# Drop unrelated comlumns "stat", "enneagram", "img_url"
df_imdb_map = df_imdb_map.drop(["overview","crew","orig_title","status","orig_lang"],axis = 1)

# Drop duplicated rows
df_imdb_map = df_imdb_map.drop_duplicates()

# Check if there is any NaN value in "names" columns 
print(df_imdb_map["names"].isna().any())

# Make sure first letters are capitalized in "names" and "genre" columns
df_imdb_map["names"] = df_imdb_map["names"].str.title()
df_imdb_map["genre"] = df_imdb_map["genre"].str.title()

# Strip whitespace
df_imdb_map['date_x'] = df_imdb_map['date_x'].str.strip()

# Convert "date_x" column to datetime
df_imdb_map['date_x'] = pd.to_datetime(df_imdb_map['date_x'], format='%m/%d/%Y')

# Create "release_year" column
df_imdb_map['release_year'] = df_imdb_map['date_x'].dt.year.astype('float')

# Rename columns to have consistency in all data sheets
df_imdb_map.rename(columns={'names': 'movie_name', 'date_x': 'release_date','budget_x':'budget'}, inplace=True)

df_imdb_map.to_csv("./data/cleaned/df_imdb_map.csv")

df_imdb_map.info()

False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10178 entries, 0 to 10177
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   movie_name    10178 non-null  object        
 1   release_date  10178 non-null  datetime64[ns]
 2   score         10178 non-null  float64       
 3   genre         10093 non-null  object        
 4   budget        10178 non-null  float64       
 5   revenue       10178 non-null  float64       
 6   country       10178 non-null  object        
 7   release_year  10178 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 715.6+ KB


In [5]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

df_imdb_map.sample(10)

,movie_name,release_date,score,genre,budget,revenue,country,release_year
2074,A Nun'S Curse,2020-05-12,64.0,Horror,92600000.0,3.533920e+08,US,2020.0
2342,Asking For It,2022-03-04,58.0,"Action, Thriller, Drama",104300000.0,1.950763e+08,US,2022.0
9238,Gintama 2: Rules Are Made To Be Broken,2018-09-13,73.0,"Comedy, Action, Science Fiction, Fantasy",89800000.0,4.128376e+08,AU,2018.0
598,Toy Story 3,2010-06-24,78.0,"Animation, Family, Comedy",200000000.0,1.068880e+09,AU,2010.0
5574,A Wrinkle In Time,2018-03-29,51.0,"Adventure, Science Fiction, Family, Fantasy",103000000.0,1.332145e+08,AU,2018.0
4772,Racing Stripes,2005-01-06,59.0,"Comedy, Family",30000000.0,8.980943e+07,US,2005.0
2401,Paw Patrol: Jet To The Rescue,2020-09-10,70.0,"Family, Animation",77600000.0,6.955130e+08,AU,2020.0
6313,The World'S End,2013-08-01,68.0,"Comedy, Action, Science Fiction",20000000.0,4.712464e+07,AU,2013.0
8180,Hotel Artemis,2018-11-07,60.0,"Thriller, Science Fiction, Action",15000000.0,1.331663e+07,AU,2018.0
4642,Aloha Scooby-Doo!,2005-02-08,72.0,"Mystery, Family, Animation, Adventure, Comedy",128000000.0,6.230386e+08,AU,2005.0


imdb_db

In [6]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

#imdb_db data import
df_db = pd.read_csv("./data/raw/imdb_db.csv")

# Drop unrelated comlumns "Number of Votes","Time Duration (min)","Director","Actors","Restriction","Description","Serie Name","Serie Date"
df_db = df_db.drop(["Number of Votes","Time Duration (min)","Director","Actors","Restriction","Description","Serie Name","Serie Date"],axis = 1)

# Drop duplicated rows
df_db = df_db.drop_duplicates()

# Check if there is any NaN value in "Movie Name" columns 
print(df_db["Movie Name"].isna().any())

# Make sure first letters are capitalized in "Movie Name" and "Movie Type" columns
df_db["Movie Name"] = df_db["Movie Name"].str.title()
df_db["Movie Type"] = df_db["Movie Type"].str.title()

# Remove square brackets in "Movie Type" column
df_db['Movie Type'] = df_db['Movie Type'].str.replace(r'[\[\]]', '', regex=True)

# Remove single quotes in "Movie Type" column
df_db['Movie Type'] = df_db['Movie Type'].str.replace(r"'", '', regex=True)

# Rename columns to have consistency in all data sheets
df_db.rename(columns={'Movie Name': 'movie_name',
                            'Movie Date': 'release_year',
                            'Movie Type':'genre',
                            'Movie Revenue (M$)':'revenue',
                            'Score':'score',
                            'Metascore':'metascore'}, inplace=True)

df_db.to_csv("./data/cleaned/df_db.csv")
df_db.info()

False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 130482 entries, 0 to 189895
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   movie_name    130482 non-null  object 
 1   release_year  130477 non-null  float64
 2   genre         130482 non-null  object 
 3   revenue       9082 non-null    float64
 4   score         130482 non-null  float64
 5   metascore     8672 non-null    float64
dtypes: float64(4), object(2)
memory usage: 7.0+ MB


In [7]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

df_db.sample(10)

,movie_name,release_year,genre,revenue,score,metascore
29866,Hermano,2010.0,"Drama, Family, Sport",29828.0,7.3,NaN
131431,Illusion Fatale,1992.0,"Drama, Thriller",NaN,6.5,NaN
152393,Traitor'S Heart,1999.0,Action,NaN,4.0,NaN
62736,Arne Dahl: Europa Blues,2012.0,"Crime, Drama, Mystery",NaN,7.3,NaN
113238,Best Of Luck,2013.0,"Action, Comedy",NaN,5.6,NaN
133705,Father Brown,2013.0,"Crime, Drama, Mystery",NaN,7.0,NaN
152938,Hellfire,1949.0,"Action, Western",NaN,7.0,NaN
140638,Bored To Death,2009.0,"Comedy, Crime, Drama",NaN,7.8,NaN
96584,One In A Billion,2016.0,Documentary,NaN,6.7,NaN
110372,Ouija Seance: The Final Game,2018.0,Horror,NaN,2.2,NaN


In [8]:
#combined_dataset = pd.merge(df_imdb_map, df_db, on=['movie_name','release_year'], how='outer')
#combined_dataset.info()

### Merge df_db and df_mbti

In [9]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

# Split df_mbti into two datasets: one where release_year is not null and one where it is null
df_mbti_with_year = df_mbti[df_mbti['release_year'].notnull()]
df_mbti_no_year = df_mbti[df_mbti['release_year'].isnull()]

# Drop release_year for merge
df_mbti_no_year.drop('release_year', inplace=True, axis=1)

# Merge the datasets based only on 'movie_name' for those without a release year
merged_no_year = pd.merge(df_mbti_no_year, df_db, on='movie_name', how='inner')

# Drop abnormal rows (there are multiple matches from the imdb_db dataset which we don't know they are correct or not)
merged_no_year.drop_duplicates(subset=['movie_name','character'],keep = 'first',inplace=True)

# Merge the datasets based on 'movie_name_clean' and 'release_year' for those with a release year
merged_with_year = pd.merge(df_mbti_with_year, df_db, on=['movie_name', 'release_year'], how='inner')

# Drop abnormal rows (there are multiple matches from the imdb_db dataset which we don't know they are correct or not)
merged_with_year.drop_duplicates(subset=['movie_name','character','release_year'],keep = 'first',inplace=True)

# Combine the two merged datasets
final_merged_df = pd.concat([merged_with_year, merged_no_year])

# Display the first few rows of the final merged dataset
final_merged_df.sort_values(['movie_name','mbti']).to_csv("./data/cleaned/df_merged.csv")

/var/folders/0n/qqmnpgh93cd_z3ptl2h4znxw0000gn/T/ipykernel_30895/1569939163.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mbti_no_year.drop('release_year', inplace=True, axis=1)


In [10]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2811 entries, 0 to 2646
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mbti          2811 non-null   object 
 1   character     2811 non-null   object 
 2   movie_name    2811 non-null   object 
 3   release_year  2811 non-null   float64
 4   genre         2811 non-null   object 
 5   revenue       384 non-null    float64
 6   score         2811 non-null   float64
 7   metascore     483 non-null    float64
dtypes: float64(4), object(4)
memory usage: 197.6+ KB


In [11]:
# Coded by Xiatong. Reviewed, tested and corrected by Daniel, Yitian, and Sai Nandini.

final_merged_df.sample(10)

,mbti,character,movie_name,release_year,genre,revenue,score,metascore
1117,INTJ,Dick Cheney,Vice,2013.0,"Documentary, News",NaN,8.9,NaN
2177,ISFP,Brett Blackmore,The F**K-It List,2020.0,Comedy,NaN,5.1,NaN
1804,ISFP,Phil,Madagascar,2005.0,"Animation, Action, Adventure",NaN,6.7,NaN
1321,ESFJ,Amy Squirrel,Bad Teacher,2014.0,Comedy,NaN,5.2,NaN
2063,ENFP,President Will Cooper,Pixels,2010.0,"Animation, Short, Action",NaN,7.4,NaN
34,ESFJ,Mamá Coco,Coco,2017.0,"Drama, Musical",NaN,5.3,54.0
1329,ESFJ,Hong-Goo,Be With You,2004.0,"Drama, Fantasy, Romance",NaN,7.9,NaN
343,ENFP,Luigi,Cars,2006.0,"Action, Sport",NaN,7.0,NaN
2270,ISTJ,Detective Cromwell,Killer Bean Forever,2009.0,"Animation, Action, Comedy",NaN,6.5,NaN
501,ISFP,Neela,The Fast And The Furious,1954.0,"Crime, Drama, Mystery",250000.0,5.4,NaN


In [12]:
# Coded by Sai Nandini. Reviewed, tested and corrected by Daniel, Yitian, and Xiatong.

final_merged_df.head()

,mbti,character,movie_name,release_year,genre,revenue,score,metascore
0,ESFP,Jack Dawson,Titanic,1997.0,"Drama, Romance",659325379.0,7.8,75.0
1,ESTP,Brock Lovett,Titanic,1997.0,"Drama, Romance",659325379.0,7.8,75.0
2,ESTP,Fifth Officer Harold Lowe,Titanic,1997.0,"Drama, Romance",659325379.0,7.8,75.0
3,ESTJ,"Caledon ""Cal"" Hockley",Titanic,1997.0,"Drama, Romance",659325379.0,7.8,75.0
4,ESTJ,Bruce Ismay,Titanic,1997.0,"Drama, Romance",659325379.0,7.8,75.0


In [13]:
# Coded by Sai Nandini. Reviewed, tested and corrected by Daniel, Yitian, and Xiatong.

# Converting the final dataset into csv file
final_merged_df.to_csv("final_merged_dataset.csv", index = False)
